In [18]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import label_binarize

In [19]:
df = pd.read_csv('../data/aden_golfe.csv',sep=';')

In [20]:
max_quinzaine=24
print df.head(3)

   Annee (0)  jour (0)  mois (0)  dist �_ la c��te (nq) zone lat (0)  long(0)  \
0       2010         1         1                    NaN  BCD  10:47N  061:37E   
1       2010         3         3                    NaN  BCD  05:14S  042:37E   
2       2010         5         3                    NaN  BCD  09:35S  044:18E   

           type(0)          pavillon statut de l'attaque(0)      ...       \
0  Vehicle Carrier    United Kingdom               Steaming      ...        
1   Fishing Vessel             Kenya               Steaming      ...        
2  Chemical Tanker  Marshall Islands               Steaming      ...        

  quinzaine_week                                      newDescriptif  Guns  \
0              1  pirates hijacked a ship underway and took her ...     0   
1              5  eleven pirates in three skiffs armed with guns...     1   
2              5  armed pirates attacked and hijacked a tanker u...     0   

   mousson  distance au port         new_type  large_area

In [21]:
def monthCorrect(month):
    month=str(int(month))
    if len(month)==1:
        return '0'+month
    else:
        return month

def fortnight(day):
    if int(day) <15:
        return 1
    else:
        return 2

def activity(years):
    if int(years) <2013:
        return 2
    else:
        return 1
    

In [22]:
df["mois (0)"]=df.apply(lambda row: monthCorrect(row['mois (0)']), axis=1)
df["jour (0)"]=df.apply(lambda row: monthCorrect(row['jour (0)']), axis=1)
df["date"]=df.apply(lambda row: str(row["Annee (0)"]) +"-"+ str(row["mois (0)"]), axis=1)
df["fortnight"]=df.apply(lambda row: fortnight(row["jour (0)"]), axis=1)
df["activity"]=df.apply(lambda row: fortnight(row["Annee (0)"]), axis=1)

#  

# Version 1 : group by years, month, area

In [23]:
df.head()

,Annee (0),jour (0),mois (0),dist �_ la c��te (nq),zone,lat (0),long(0),type(0),pavillon,statut de l'attaque(0),...,mousson,distance au port,new_type,large_area,lat_area,lon_area,area_by_hand,date,fortnight,activity
0,2010,01,01,NaN,BCD,10:47N,061:37E,Vehicle Carrier,United Kingdom,Steaming,...,0,1067.512949,Vehicle Carrier,A,NaN,NaN,AA,2010-01,1,2
1,2010,03,03,NaN,BCD,05:14S,042:37E,Fishing Vessel,Kenya,Steaming,...,0,353.541684,Vessel,I,NaN,NaN,AP,2010-03,1,2
2,2010,05,03,NaN,BCD,09:35S,044:18E,Chemical Tanker,Marshall Islands,Steaming,...,0,455.926794,Tanker,I,NaN,NaN,AP,2010-03,1,2
3,2010,23,03,NaN,BCD,11:41.53N,066:05.38E,Bulk Carrier,Malta,Steaming,...,0,929.851019,Vehicle Carrier,A,NaN,NaN,AC,2010-03,2,2
4,2010,23,03,NaN,BCD,17:28N,056:42E,Refrigerated Cargo,Bermuda,Steaming,...,0,291.087286,Vehicle Carrier,A,NaN,NaN,V,2010-03,2,2


In [24]:
print df["area_by_hand"].unique()

['AA' 'AP' 'AC' 'V' 'Z' 'AF' 'AL' 'AM' 'AG' 'AJ' 'W' 'M' 'AB' 'L' 'X' 'Y'
 'S' 'AN' 'H' 'I' 'G' 'J' 'C' 'AI' 'AQ' 'T' 'AO' 'D' 'F' 'K' 'E' 'A' 'B'
 'AH' 'AE' 'R' 'N' 'O' 'P' 'AR']


In [25]:
dfAgregateByArea=pd.DataFrame({'count' : df.groupby( [ "Annee (0)","mois (0)","area_by_hand","fortnight","new_type"] ).size()}).reset_index()

In [26]:
print dfAgregateByArea.head(10)
print dfAgregateByArea["count"].sum()

   Annee (0) mois (0) area_by_hand  fortnight         new_type  count
0       2008       02           AL          1  Vehicle Carrier      1
1       2008       02           AP          1            Other      1
2       2008       02            K          1           Tanker      1
3       2008       02            K          1              Tug      1
4       2008       03           AI          1            Other      1
5       2008       03           AP          1            Other      1
6       2008       03           AP          2            Other      1
7       2008       03            J          2           Tanker      1
8       2008       03            K          2  Vehicle Carrier      1
9       2008       03            M          2           Vessel      1
883


### Creation empty dataframe

In [27]:
uniqueArea=sorted(dfAgregateByArea["area_by_hand"].unique())
uniqueYears=dfAgregateByArea['Annee (0)'].unique()
uniqueTypeBoat=dfAgregateByArea['new_type'].unique()
uniqueMonth=np.linspace(1, 12, num=12)
uniqueBinary=np.linspace(1, 2, num=2)



ColYears=np.repeat(uniqueYears,len(uniqueMonth)*len(uniqueTypeBoat)*len(uniqueArea)*len(uniqueBinary))
ColMonth=np.tile(np.repeat(uniqueMonth,len(uniqueArea)*len(uniqueBinary)*len(uniqueTypeBoat)), len(uniqueYears))
ColArea=np.tile(uniqueArea,len(uniqueMonth)*len(uniqueTypeBoat)*len(uniqueBinary)*len(uniqueYears))

ColType=np.tile(np.repeat(uniqueTypeBoat,len(uniqueArea)*len(uniqueBinary)),len(uniqueMonth)*len(uniqueYears))

ColFortnight=np.tile(np.repeat(uniqueBinary, len(uniqueArea)),len(uniqueMonth)*len(uniqueTypeBoat)*len(uniqueYears))
ColCount=np.repeat(0, len(ColFortnight))


print ColYears[0:100]
print ColMonth[0:200]
print ColArea[0:200]
print ColType[0:200]

print len(ColYears)
print len(ColMonth)
print len(ColArea)
print len(ColType)

print len(ColFortnight)


[2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008
 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008
 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008
 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008
 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008
 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008
 2008 2008 2008 2008 2008 2008 2008 2008 2008 2008]
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  

In [28]:
dic = {'Years': ColYears,'Month': ColMonth,'Fortnight': ColFortnight, 'Area': ColArea,'Type':ColType,'Count': ColCount}
dfFinalArea= pd.DataFrame(dic)
print dfFinalArea.head(50)

   Area  Count  Fortnight  Month             Type  Years
0     A      0          1      1  Vehicle Carrier   2008
1    AA      0          1      1  Vehicle Carrier   2008
2    AB      0          1      1  Vehicle Carrier   2008
3    AC      0          1      1  Vehicle Carrier   2008
4    AE      0          1      1  Vehicle Carrier   2008
5    AF      0          1      1  Vehicle Carrier   2008
6    AG      0          1      1  Vehicle Carrier   2008
7    AH      0          1      1  Vehicle Carrier   2008
8    AI      0          1      1  Vehicle Carrier   2008
9    AJ      0          1      1  Vehicle Carrier   2008
10   AL      0          1      1  Vehicle Carrier   2008
11   AM      0          1      1  Vehicle Carrier   2008
12   AN      0          1      1  Vehicle Carrier   2008
13   AO      0          1      1  Vehicle Carrier   2008
14   AP      0          1      1  Vehicle Carrier   2008
15   AQ      0          1      1  Vehicle Carrier   2008
16   AR      0          1      

### Fill dataframe

In [29]:
for index, row in dfAgregateByArea.iterrows():
    #print "---------------------------------------------------------" 
    
    idx=dfFinalArea[(dfFinalArea['Years']==row['Annee (0)']) & (dfFinalArea['Area']==row['area_by_hand']) 
                    & (dfFinalArea['Type']==row['new_type'])& (dfFinalArea['Fortnight']==row['fortnight'])
                    & (dfFinalArea['Month']==int(row["mois (0)"]))]
    #print len(idx)
    dfFinalArea.set_value(idx.index.values, "Count",row['count'])

#print dfFinalArea.head(40)
#print dfFinalArea["Count"].sum()

In [30]:
def getActivity(Years):
    activity=1
    if Years<2013:
        activity=2     
    return activity

dfFinalArea["Activity"]=dfFinalArea.apply(lambda row: getActivity(row['Years']), axis=1)
print dfFinalArea.head()
print len(dfFinalArea)

  Area  Count  Fortnight  Month             Type  Years  Activity
0    A      0          1      1  Vehicle Carrier   2008         2
1   AA      0          1      1  Vehicle Carrier   2008         2
2   AB      0          1      1  Vehicle Carrier   2008         2
3   AC      0          1      1  Vehicle Carrier   2008         2
4   AE      0          1      1  Vehicle Carrier   2008         2
60480


In [31]:
dfFinalArea.to_csv("../data/aden_golfe_matrix_area_hand.csv",sep=';', encoding='utf-8', index=False)

# Version 2 : group by Date, type attack

In [6]:
dfAgregateTotalType=pd.DataFrame({'count' : df.groupby( [ "date","new_type"] ).size()}).reset_index()
dfAgregateTotalType["date"]=dfAgregateTotalType["date"]+"-01"
print dfAgregateTotalType.head(10)

         date         new_type  count
0  2008-02-01           Tanker      1
1  2008-02-01              Tug      1
2  2008-03-01           Tanker      1
3  2008-03-01  Vehicle Carrier      1
4  2008-03-01           Vessel      1
5  2008-04-01           Tanker      2
6  2008-04-01  Vehicle Carrier      2
7  2008-04-01           Vessel      2
8  2008-05-01            Other      1
9  2008-05-01           Tanker      1


In [7]:
max_month=12
unique_years=sorted(df["Annee (0)"].unique())
unique_type=sorted(dfAgregateTotalType["new_type"].unique())
month_repeat=np.linspace(1, max_month, num=max_month)

colYears=np.repeat(unique_years, max_month)
colMonth=np.tile(month_repeat, len(unique_years))
                               
dic2 = {'Years': colYears, 'Month': colMonth}
dfGlobale = pd.DataFrame(dic2)

for i in range(0,len(unique_type)):
    nameColLat=str(unique_type[i])
    dfGlobale[nameColLat]=0


dfGlobale.head()
dfGlobale["Month"]=dfGlobale.apply(lambda row: monthCorrect(row['Month']), axis=1)
dfGlobale["date"]=dfGlobale.apply(lambda row: str(row["Years"]) +"-"+ str(row["Month"])+"-01", axis=1)
dfGlobale = dfGlobale.drop('Month', 1)                            
dfGlobale = dfGlobale.drop('Years', 1)
print dfGlobale.head()

   Barge  Offshore Exploitation  Other  Tanker  Tug  Vehicle Carrier  Vessel  \
0      0                      0      0       0    0                0       0   
1      0                      0      0       0    0                0       0   
2      0                      0      0       0    0                0       0   
3      0                      0      0       0    0                0       0   
4      0                      0      0       0    0                0       0   

         date  
0  2008-01-01  
1  2008-02-01  
2  2008-03-01  
3  2008-04-01  
4  2008-05-01  


# Fill matrice

In [8]:
for index, row in dfAgregateTotalType.iterrows():
    #print 'Years '+str(row['Annee (0)']),'2_week : '+str( row['quinzaine_week']),'count : '+str(row['count'])#,'type_attack : '+str(row['type d\'attaque(0)'])

    idx=dfGlobale[(dfGlobale['date']==row['date'])]
    dfGlobale.set_value(idx.index.values, str(row["new_type"]),row['count'])

print dfGlobale.head(100)

    Barge  Offshore Exploitation  Other  Tanker  Tug  Vehicle Carrier  Vessel  \
0       0                      0      0       0    0                0       0   
1       0                      0      0       1    1                0       0   
2       0                      0      0       1    0                1       1   
3       0                      0      0       2    0                2       2   
4       0                      0      1       1    0                4       0   
5       0                      0      1       0    0                1       0   
6       0                      0      1       1    0                2       0   
7       0                      0      0       3    0                6       0   
8       1                      0      2       7    0                9       0   
9       0                      0      2       7    0                7       0   
10      0                      0      1       8    0                6       1   
11      0                   

In [9]:
dfGlobale.to_csv("../data/aden_golfe_matrix_global.csv",sep=';', encoding='utf-8', index=False)